# PX912: Solid Mechanics

## Workshop 4

Last week, we considered a simple finite element problem in 1D. This week, instead we will look at a 2D problem of load applied to a trapezoidal panel fixed to a wall. I have tried to break the problem down into smaller steps to help you understand the process.

### Please run the cell below!
This cell loads the core library written for this module. The core library contains hints, solution checking and grading. 

Make sure that the output of the previous cell is $\texttt{Library Loaded!}$. 

In [ ]:
import sys, os
sys.path.insert(0, os.getcwd()+'grader')
from grader import workshop4 as ws4
from grader import practice

grader = ws4.Workshop4()

Imports as usual:

In [ ]:
# SymPy Library: Symbolic Python
import sympy as sym

# Tell sympy to print things nicely
sym.init_printing()

# Matplotlib for plotting
import matplotlib.pyplot as plt

# Numpy for numerics
import numpy as np

## A simplified summary of the Finite Element Method
FEM is a masterclass in numerical analysis. The individual components appear disparate, but fit together to form a powerful technique. Let's walk through the finite element method approach, starting from the very fundamentals.

### The strong form
A problem is first derived in terms of a differential equation. For an elastic bar of length $l$ in one dimension, one would write the governing equation as
$$
\frac{\text{d}}{\text{d}x}\left( A E \frac{\text{d}u}{\text{d}x} \right) + b = 0,
$$
with boundary conditions being
$$
\sigma(0) = \left( E \frac{\text{d}u}{\text{d}x}\right) = -\bar{t} \\
u(l) = \bar{u}.
$$

Here, $A$ and $E$ are material parameters, $u$ is the displacement, $b$ is the body force, $\bar{t}$ is the traction boundary condition and $\bar{u}$ is the prescribed displacement.

### The weak form
You can actually solve the above equations using a simple finite element approach as it is (see *Fish and Belytschko*). However, for multidimensional problems and more complex elements, this will not work. For this we motivate the *weak form* of the differential equation. The problem becomes the following:

Find $u(x)$ among the smooth trial solutions $u\in U$ such that 
$$
\left. \int_0^l \left(\frac{\text{d}w}{\text{d}x}\right)^T AE \frac{\text{d}u}{\text{d}x} \text{d}x - \int_0^l w^T b \text{d}x - w^T \bar{t}A) \ \right|_{ \ x=0} = 0 
\qquad
\forall w(x) \text{ satisfying some boundary condition.}
$$
The weight (or test) functions $w$ are completely arbitrary: this property is what is used to reduce the system into a set of linear-algebraic equations. This is a result from the theory of differential equations, where the solution of a differential equation may be constructed from a *complete* basis set. 

### Element interpolation
The finite element method operates on nodes, we do not use continuous functions. We need to find a way of representing these functions in terms of matrices. We do so using *Lagrange interpolation*. This method operates by representing a function in terms of polynomials,

$$
u^e(x) = a_0 + a_1 x + a_2 x^2 + \ ...
$$
Let's write this as 
$$
u^e(x) = 
\left[ \begin{matrix}1 & x & x^2 & ... \end{matrix} \right]
\left[ \begin{matrix}a_0 \\ a_1 \\ a_2 \\ ... \end{matrix} \right]
= 
\mathbf{p}(x)\mathbf{a}^e
$$

Then, for multiple defined points, 
$$
u^e(x_0^e) = a_0 + a_1 x_0^e + a_2 (x_0^e)^2 + \ ... \\
u^e(x_1^e) = a_0 + a_1 x_1^e + a_2 (x_1^e)^2 + \ ... \\
u^e(x_2^e) = a_0 + a_1 x_2^e + a_2 (x_2^e)^2 + \ ... ,
$$
the system can be written as 
$$
\left[ \begin{matrix} u^e_0 \\ u^e_1 \\ u^e_2 \\ ... \end{matrix} \right]
=
\left[ \begin{matrix} 
1 & x_0^e & (x_0^e)^2 & ... \\
1 & x_1^e & (x_1^e)^2 & ... \\
1 & x_2^e & (x_2^e)^2 & ... \\
... & ... & ... & ...
\end{matrix} \right]
\left[ \begin{matrix} a_0 \\ a_1 \\ a_2 \\ ... \end{matrix} \right].
$$
We can represent this system like so,
$$
\mathbf{d}^e = \mathbf{M}^e \mathbf{a}^e.
$$
Then,
$$
\mathbf{a}^e = (\mathbf{M}^e)^{-1} \mathbf{d}^e,
$$ 

and so 
$$
u^e(x) = \mathbf{p}(x) (\mathbf{M}^e)^{-1}\mathbf{d}^e  = \mathbf{N}^e\mathbf{d}^e
$$

Here, $\mathbf{N}$ is a shape function. It plays a central role in FEM and can also be used to treat derivatives. Considering a two-node system where nodes are labelled with subscripts 1 and 2,
$$
\mathbf{N}^e = 
\left[ \begin{matrix} N_1^e & N_2^e \end{matrix} \right] =
\mathbf{p}(x)(\mathbf{M}^e)^{-1} =
\left[ \begin{matrix} 1 & x \end{matrix} \right]
\left[ \begin{matrix} x_2^e & -x_1^e \\ -1 & 1 \end{matrix} \right]\frac{1}{l^e} = 
\left[ \begin{matrix} x_2^e - x & x - x_1^e \end{matrix} \right]
$$

Note that $l^e = x_2^e - x_1^e$ and is the length of an element that emerges from the calculation of $\left[\mathbf{M}^e\right]^{-1}$ as the determinant. From this, the derivative of the shape-function can be found like so
$$
\mathbf{B}^e = 
\left[ \begin{matrix} \frac{\text{d}N_1^e}{\text{d}x} & \frac{\text{d}N_2^e}{\text{d}x} \end{matrix} \right] = 
\frac{1}{l^e}\left[ \begin{matrix} -1 & 1 \end{matrix} \right]
$$

Using these shape functions, we can treat the weak form of the system in the previous section. As just calculated, for trial functions $u(x)$, 
$$
u^e \approx \mathbf{N}^e\mathbf{d}^e.
$$

Likewise, using the same interpolation for our weight functions $w(x)$ (a method known as Galerkin FEM), 
$$
w^e \approx \mathbf{N}^e\mathbf{w}^e.
$$

The global solution (for when you have more than one element in your system) can then be found by simply adding together every weight function and trial solution per element.
$$
u(x) \approx u^h = \sum_{e=1}^{n_{el}} \mathbf{N}^e \mathbf{d}^e =
\left( \sum_{e=1}^{n_{el}} \mathbf{N}^e \mathbf{L} \right)\mathbf{d} = \mathbf{N}(x) \mathbf{d} \\
w(x) \approx w^h = \sum_{e=1}^{n_{el}} \mathbf{N}^e \mathbf{w}^e = 
\left( \sum_{e=1}^{n_{el}} \mathbf{N}^e \mathbf{L} \right)\mathbf{w} = \mathbf{N}(x) \mathbf{w}
$$
Here, $\mathbf{L}$ is the scatter matrix that maps a local node to the global system and $\mathbf{d}$ and $\mathbf{w}$ are the global displacement and weight vectors.

### System equations
Let's substitute. We start from  
$$
\left. \int_0^l \left(\frac{\text{d}w}{\text{d}x}\right)^T AE \frac{\text{d}u}{\text{d}x} \text{d}x - \int_0^l w^T b \text{d}x - w^T \bar{t}A) \ \right|_{ \ x=0} = 0,
$$

which is the continuous formulation. If we want to break this down into elements, we have to integrate over each element
$$
\sum_{e=1}^{n_{el}}
\left\{
\left. \int_{x_1^e}^{x_2^e} \left(\frac{\text{d}w^e}{\text{d}x}\right)^T A^e E^e \frac{\text{d}u^e}{\text{d}x} \text{d}x - \int_{x_1^e}^{x_2^e} w^{eT} b \text{d}x - w^{eT} \bar{t}A^e) \ \right|_{ \ x=0}
\right\} = 0.
$$
Then, when 
$$
u^e(x) = \mathbf{N}^e\mathbf{d}^e \qquad \frac{\text{d}u^e}{\text{d}x} = \mathbf{B}^e\mathbf{d}^e \\
w^{eT}(x) = \mathbf{N}^{eT}\mathbf{w}^{eT} \qquad \left(\frac{\text{d}w^e}{\text{d}x}\right)^T = \mathbf{d}^{eT}\mathbf{B}^{eT},
$$
the full system coalesces into a signficantly more tractable

$$
\sum_{e=1}^{n_{el}}
\mathbf{w}^{eT}
\left\{
\int_{x_1^e}^{x_2^e} \mathbf{B}^{eT}A^e E^e \mathbf{B}^e \text{d}x \mathbf{d}^e - 
\int_{x_1^e}^{x_2^e} \mathbf{N}^{eT} b \text{d}x - (\mathbf{N}^{eT} A^e \bar{t})_{x=0}
\right\}
$$

This can be simplified into what might seem like a familiar form:

$$
\sum_{e=1}^{n_{el}}
\mathbf{w}^{eT}
\left\{
\mathbf{K}^e - \mathbf{f_b}^e - \mathbf{f_{\text{boundary}}}
\right\} =
\sum_{e=1}^{n_{el}}
\mathbf{w}^{eT}
\left\{
\mathbf{K}^e - \mathbf{f_e}^e
\right\} = 0
$$
where $\mathbf{K}^e$ is the element stiffness matrix and the latter two force calculations constitute the element external stress matrix $\mathbf{f}^e$.

Now we need to assemble the full system. Using the assembly operation (which we denote for each element using $\mathbf{L}^e$), the above equations can be further simplified to 
$$
\mathbf{w}^T 
\left( 
\sum_{e=1}^{n_{el}}
\mathbf{L}^{eT} \mathbf{K}^e \mathbf{L}^{e} \mathbf{d}
-
\sum_{e=1}^{n_{el}}
\mathbf{L}^{eT}
\mathbf{f}^e
\right).
$$
From here, the assembled matrices can be written. The system stiffness matrix is
$$
\mathbf{K} = \sum_{e=1}^{n_{el}}
\mathbf{L}^{eT} \mathbf{K}^e \mathbf{L}^{e},
$$
whilst the system external force matrix is 
$$
\mathbf{f} = 
\sum_{e=1}^{n_{el}}
\mathbf{L}^{eT}
\mathbf{f}^e
$$

(You don't need to use $\mathbf{L}^e$ as a linear-algebraic operator at all: the normal way you've developed using array manipulations is more efficient).

Our problem is thus reduced to:
$$
\mathbf{w}^T (\mathbf{K}\mathbf{d} - \mathbf{f}) = 0 \qquad \forall w(x) \text{ satisfying some boundary condition.}
$$

For more detail, please refer to your lecture notes.

### Two dimensional considerations
The above treatment is in one dimension. To consider two-dimensional problems, one must use *isoparametric elements*. 
* There are now twice as many degrees of freedom. This will affect the sizes of your vectors and matrices.
* Interpolation occurs between multiple varibles.
* Linear differential operators are now replaced with gradient operators (in two dimensions).


## Question 1
Note that in the previous treatment, definitions for $\mathbf{K}^e$ and $\mathbf{f}^e$ require integration. For most problems, this integration process must be carried out numerically. Here we'll use Gauss quadrature in order to do this.

Implement Gauss quadrature to obtain exact values for the following
integrals, and verify by analytical integration.

**a)** 
$$
I_a = \int^{1}_{-1} \left( \xi^4 + 2 \xi^2 \right) \text{d}\xi
$$

In [ ]:
numerical_a = ...
    
print('Numerical result is: ', numerical_a)

exact_a = ...

print('Exact result is: ', exact_a)

In [ ]:
# HINT AND SOLUTION
# grader.hint1a()
grader.check1a(numerical_a, exact_a)

**b)** 
$$
I_b = \int^{4}_{0} \left( x^2 + 1 \right) \text{d}x
$$

In [ ]:
numerical_b = ...

print('Numerical result is: ', numerical_b)

exact_b = ...

print('Exact result is: ', exact_b)

In [ ]:
# HINT AND SOLUTION
# grader.hint1b()
grader.check1b(numerical_b, exact_b)

## Question 2
We consider a linear elasticity problem on a trapezoidal panel domain, as shown below:

<img src="grader/pictures/workshop4.png" alt="Drawing" style="width: 800px;"/>

The panel is made of a material with Young’s modulus
$E =30$ MPa and Poisson’s ratio $\nu=0.3$. Plane stress conditions are
assumed. The problem is discretised with a linear quadrilateral element.
Calculate strains and stresses at element integration (Gauss) points using
the finite-element approach.

### Inputs

First, specify the elasticity matrix for this problem, using the notes to help.

In [ ]:
def Plane_Stress_C(E, nu):
    ...

In [ ]:
E = 30e6
nu = 0.3

ps = Plane_Stress_C(E, nu)
print(ps)

In [ ]:
# HINT AND SOLUTION
# grader.hint2ai()
grader.check2ai(ps)

Identify the **fixed displacements** within the global displacement vector. Which indices of the vector below will have zero value?

$$
\mathbf{d}^e = [d^e_{1(1)}, d^e_{2(1)}, d^e_{1(2)},d^e_{2(2)}, d^e_{1(3)}, d^e_{2(3)}, d^e_{1(4)}, d^e_{2(4)}] 
$$

In [ ]:
# YOUR CODE HERE
fixed_disp = ...

In [ ]:
# HINT AND SOLUTION
# grader.hint2aii()
grader.check2aii(fixed_disp)

We will also need the nodal force vector, which takes the form.

$$
\mathbf{f}^e = [f^e_{1(1)}, f^e_{2(1)}, f^e_{1(2)},f^e_{2(2)}, f^e_{1(3)}, f^e_{2(3)}, f^e_{1(4)}, f^e_{2(4)}] 
$$
This we will specify later.

Let's get the coordinate matrix, or the mapping between the normalised coordinates and the real coordinates. When running the real calculation, you probably don't need a function to automatically translate points. However, you need the mapping function to calculate important quantities - it is always useful to put it in function form.

In [ ]:
def xf(xi, eta):
    ...

def yf(xi, eta):
    ...

In [ ]:
print(xf(-1,-1), yf(-1,-1))
print(xf(1,-1), yf(1,-1))
print(xf(-1,1), yf(-1,1))
print(xf(1,1), yf(1,1))

In [ ]:
# HINT AND SOLUTION
# grader.hint2aiii()
grader.check2aiii(xf, yf)

Now let's specify the shape functions. There should be one for each node

In [ ]:
# YOUR CODE HERE
# you might like to write a function that calculates all of these at once
N_1 = 0
N_2 = 0
N_3 = 0
N_4 = 0

In [ ]:
# HINT
# grader.hint2aiv()
grader.check2aiv(N)

### Finite Element Matrices

We should now specify the strain-displacement matrix

$$
\mathbf{B}^e = [\mathbf{B}^e_1, \mathbf{B}^e_2, \mathbf{B}^e_3, \mathbf{B}^e_4]
$$

In [ ]:
def Disp_Strain_B(real_coords, xi, eta):
    ...

In [ ]:
# HINT
# grader.hint2bi()

And the Jacobian:

$$
\mathbf{J}^e =
\begin{bmatrix}
\frac{\partial x_1}{\partial \xi} & \frac{\partial x_2}{\partial \xi} \\
\frac{\partial x_1}{\partial \eta} & \frac{\partial x_2}{\partial \eta}
\end{bmatrix}
$$

In [ ]:
def J(xi, eta):
    ...

def J_inv(xi, eta):
    ...

In [ ]:
# HINT
# grader.hint2bii()

Then at each integration point we need to build the stiffness matrix from the Jacobian, the strain-displacement matrix, and the elasticity matrix:

$$
\mathbf{K}^e(\xi_i, \eta_i) = {\mathbf{B}^e}^T(\xi_i, \eta_i)) \mathbf{C} \mathbf{B}^e(\xi_i, \eta_i) \det(\mathbf{J}^e(\xi_i, \eta_i)
$$

In [ ]:
def keval(real_coords, Ce):
    ...

In [ ]:
# HINT
# grader.hint2biii()

These should then be assembled into the elemental stiffness matrix:

$$
\mathbf{K}^e = \sum_i \sum_j \mathbf{K}(\xi_i, \eta_j)
$$

In [ ]:
# YOUR CODE HERE
#Boundary conditions
coords = ...

# Calculate element stiffness matrix
C = ...
ke = ...

# Remove components of the stiffness matrix that correspond to zero DOFs

print(ke)

In [ ]:
# HINT
grader.check2biii(ke)

Almost there! Now we add the external loads into the nodal force vector:

$$
\mathbf{f}^e = \int_{\Gamma_{14}}{\mathbf{N}^e}^T \bar{\mathbf{t}} \, \text{d}\Gamma
$$

In [ ]:
# Applied loads - define the RHS load vector 
fext_top = ...

# create RHS vector
f_e = ...

In [ ]:
# HINT
# grader.hint2biv()
grader.check2biv(f_e)

### Solution

Now we need to reduce the overall system of equations and solve for the displacements.

In [ ]:
# Calculate non-zero nodal displacements
ue = ...

# Create the global nodal displacement vector 
d_e = ...
print(d_e)

In [ ]:
# HINT
# grader.hint2ci()
grader.check2ci(d_e)

In [ ]:
# Plot element deformation (nodal displacements)
sdef=10000

plt.plot(coords[:, 0],
         coords[:, 1], 'sk')
plt.plot(coords[:, 0] + sdef*d_e[0:2*len(coords)-1:2],
         coords[:, 1] + sdef*d_e[1:2*len(coords):2], 'or')

for i in range(len(coords)):
    plt.fill(coords[:, 0],
             coords[:, 1], edgecolor='k', fill=False)
    plt.fill(coords[:, 0] + sdef*d_e[0:2*len(coords)-1:2],
             coords[:, 1] + sdef*d_e[1:2*len(coords):2], edgecolor='r', fill=False)
plt.show()

We can then solve for the strains at the four Gauss points:
$$
\varepsilon^e(\xi_i, \eta_j) = \mathbf{B}^e(\xi_i, \eta_j) \mathbf{d}^e = [\varepsilon^e_{11}, \varepsilon^e_{22}, \gamma^e_{12}]^T
$$

In [ ]:
# YOUR CODE HERE

def strains(real_coords, de):
    ...
        
strain_vals = ...
print(strain_vals)

In [ ]:
# HINT
# grader.hint2cii()
grader.check2cii(strain_vals)

And similarly the stresses:

$$
\sigma^e(\xi_i, \eta_j) = \mathbf{C}\varepsilon^e(\xi_i, \eta_j) = [\sigma^e_{11}, \sigma^e_{22}, \sigma^e_{12}]^T
$$

Return the *sum* of stresses over the Gauss points.

In [ ]:
def sigma_vector(coords,C,de):
    ...

sigma = sigma_vector(coords,C,d_e)

print(sigma)

In [ ]:
# HINT
# grader.hint2ciii()
grader.check2ciii(sigma)

And that completes the solution to the FE problem!

# Results

Run the box below to check your progress.

In [ ]:
grader.results()